In [24]:

import json
import sqlite3
import pandas as pd
import numpy as np
import bokeh.models as bkm

from bokeh.io import curdoc
from bokeh.plotting import figure, curdoc
from bokeh.layouts import column, row
from bokeh.models import HoverTool, LabelSet, ColumnDataSource,  DataRange1d, Select, DateRangeSlider, Slider
from bokeh.tile_providers import get_provider, STAMEN_TERRAIN, OSM, CARTODBPOSITRON
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.layouts import layout

In [35]:
# SQLite database of DRONES
# db_file_drones = '/Users/gianlucafilippi/GitHub/smart-o2c/MATLAB/Problems/CAELUS/AgentBasedModel/drones_sim.db'
db_file_drones = '/home/caelus/Documents/GitHub/CAELUS_Optimisation/src/ModelsMetrics/AgentBased/Data_Bases/drones_sim.db'
# SQLite database of STATIONS
# db_file_stations = '/Users/gianlucafilippi/GitHub/smart-o2c/MATLAB/Problems/CAELUS/AgentBase/SQLlite/test_stationstype.db'
# db_file_stations = '/Users/gianlucafilippi/GitHub/smart-o2c/MATLAB/Problems/CAELUS/AgentBasedModel/stations_sim.db'
db_file_stations = '/home/caelus/Documents/GitHub/CAELUS_Optimisation/src/ModelsMetrics/AgentBased/Data_Bases/stations_sim.db'
# image url
# # image url
url_image = '/Users/gianlucafilippi/GitHub/CAELUS_DT/CAELUS_Interface_Optimiser_DT/Tracker/airplane.png'


In [36]:


#FUNCTION TO CONVERT GCS WGS84 TO WEB MERCATOR
#DATAFRAME
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

#POINT
def wgs84_web_mercator_point(lon,lat):
    k = 6378137
    x= lon * (k * np.pi/180.0)
    y= np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x,y

#AREA EXTENT COORDINATE WGS84
lon_min,lat_min=-8, 55 # -125.974,30.038
lon_max,lat_max= 0, 61 # -68.748,52.214

#COORDINATE CONVERSION
xy_min=wgs84_web_mercator_point(lon_min,lat_min)
xy_max=wgs84_web_mercator_point(lon_max,lat_max)

#COORDINATE RANGE IN WEB MERCATOR
x_range,y_range=([xy_min[0],xy_max[0]], [xy_min[1],xy_max[1]])




conn_drones = sqlite3.connect(db_file_drones)
conn_stations = sqlite3.connect(db_file_stations)


In [37]:
#PLOT AIRCRAFT POSITION
p=figure(x_range=x_range,y_range=y_range,
        x_axis_type='mercator',y_axis_type='mercator',
        sizing_mode='scale_width',plot_height=350, #width=300, #plot_height
        tools="pan, box_select, zoom_in, zoom_out, box_zoom, save, reset")

In [38]:
station_name_db = {
    'system':[],   
    'textID':[], 
    'numberID':[], 
    'info':[], 
    'type':[], 
    'lon':[], 
    'lat':[], 
    'x':[], 
    'y':[], 
    'status':[], 
    'storge_capacity':[], 
    'charging_capacity':[], 
    'infrastructure_chargin':[], 
    'infrastructure_takeoff_landing':[], 
    'infrastructure_drone_storage':[]
}
stations_source_A=ColumnDataSource(station_name_db)
stations_source_H=ColumnDataSource(station_name_db)
stations_source_L=ColumnDataSource(station_name_db)
stations_source_new=ColumnDataSource(station_name_db)

In [41]:
station_name_db

{'system': [],
 'textID': [],
 'numberID': [],
 'info': [],
 'type': [],
 'lon': [],
 'lat': [],
 'x': [],
 'y': [],
 'status': [],
 'storge_capacity': [],
 'charging_capacity': [],
 'infrastructure_chargin': [],
 'infrastructure_takeoff_landing': [],
 'infrastructure_drone_storage': []}